<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
from google.colab import userdata
import os

In [3]:
files.upload(); #Upload kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list

ref                                                   title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                    Vehicle Sales Data                              19MB  2024-02-21 20:16:17          16410        288  1.0              
sudarshan24byte/online-food-dataset                   Online Food Dataset                              3KB  2024-03-02 18:50:30           5927        108  0.9411765        
nbroad/gemma-rewrite-nbroad                           gemma-rewrite-nbroad                             8MB  2024-03-03 04:52:39            765         69  1.0              
arnavvvvv/spotify-music                               Top Spotify Songs                               47KB  2024-03-06 05:20:29        

In [6]:
!git clone https://github.com/Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 3152 (delta 447), reused 438 (delta 358), pack-reused 2544
Receiving objects: 100% (3152/3152), 16.15 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (1820/1820), done.
Cloning into 'ivy'...
remote: Enumerating objects: 188649, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 188649 (delta 301), reused 346 (delta 261), pack-reused 188200
Receiving objects: 100% (188649/188649), 164.16 MiB | 16.30 MiB/s, done.
Resolving deltas: 100% (151942/151942), done.


In [7]:
!pip install -U -q ivy accelerate>=0.21.0 mlflow datasets>=2.14.5 nlp 2>/dev/null
import ivy

In [8]:
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

 94% 204M/216M [00:01<00:00, 137MB/s]
100% 216M/216M [00:01<00:00, 117MB/s]
Archive:  /content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-balanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-unbalanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarc.csv.gz  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv  


In [9]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

import accelerate

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

In [10]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")
df = df.drop_duplicates()

In [11]:
df = df[['comment', 'label']]
df = df[~df['comment'].isnull()]
df = df[~df['label'].isnull()]
print(df.sample(15))

                                                  comment  label
270838        You just didn't want the bad poosay enough.      0
970105             That, there is no compulsion in Islam!      0
280051                             He's alright, I guess.      1
802302          no wtf we don't want that flopper lechoke      1
653077  One person said something he disagrees with an...      1
391321  I refuse to believe that prospects weren't inv...      1
254714  Or excessive levels adjustment if you want to ...      0
788912  Yeah because the first book bombed in all the ...      1
169382                     tweet about it to donald trump      0
392578                                     $470 with tax.      0
723591                            it's actually "nun is".      1
349564  Being strictly fair, they are if they were tak...      0
269778  "How dare those black people not vote how we w...      1
215175  Has to be a 1st rounder for such willfull disr...      1
331323                   

In [12]:
dataset = Dataset.from_pandas(df)
del df
gc.collect()


5

In [13]:
def get_training_corpus():
  training_corpus = (
    dataset["comment"][i : i + 1000]
    for i in range(0, len(dataset["comment"]), 1000)
  )
  return training_corpus

training_corpus = get_training_corpus()


In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer.save_pretrained("Sarcasm_Detection-Tokenizer")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
files.upload(); #Upload Sarcasm_Detection-Tokenizer.zip if already trained -> '/content/Sarcasm Detection/'

Saving Sarcasm_Detection-Tokenizer.zip to Sarcasm_Detection-Tokenizer.zip


In [15]:
!unzip '/content/Sarcasm_Detection-Tokenizer.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

Archive:  /content/Sarcasm_Detection-Tokenizer.zip
   creating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/kaggle.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/merges.txt  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/special_tokens_map.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer_config.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/vocab.json  


In [16]:
tokenizer = AutoTokenizer.from_pretrained("/content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(dataset["comment"][1], dataset["label"][1])
tokens = tokenizer(dataset["comment"][1:5], padding=True)
print(tokens)
decoded_string = tokenizer.decode(tokens.get("input_ids")[0])
print(decoded_string)

You do know west teams play against west teams more than east teams right? 0
{'input_ids': [[446, 363, 465, 5140, 2347, 528, 1095, 5140, 2347, 506, 525, 5338, 2347, 539, 31, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [741, 631, 42310, 4887, 1953, 12, 392, 1014, 18461, 312, 10837, 371, 15283, 12, 265, 9009, 1698, 515, 5277, 284, 14656, 742, 17], [677, 3195, 781, 320, 1460, 3407, 302, 265, 470, 9154, 25745, 12973, 2, 1671, 354, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [41, 609, 713, 479, 302, 646, 7910, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
You do know west teams play against west teams more

In [17]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

def get_tokenized_dataset():
  tokenized_dataset = ([tokenize_function(token) for token in batch] for batch in training_corpus)
  return tokenized_dataset

tokenized_dataset = get_tokenized_dataset()

In [18]:
i:int = 0
for batch in tokenized_dataset:
  for token in batch:
    print(token)
    i+= 1
    if i > 3:
      break
  break

{'input_ids': [5618, 307, 29473, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 5200

In [19]:
model = AutoModel.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [20]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [21]:
training_args = TrainingArguments(output_dir="/content/demos/Contributor_demos/Sarcasm Detection") #Requires enabling GPU

In [ ]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *